# Youtube 컨텐츠 새로 불러오기

In [19]:
from crawling_func.youtube_crawling import get_video_by_keys, get_product_urls
from crawling_func.shopkko_crawling import get_kko_product_info, get_kko_product_reviews
from crawling_func.shop29cm_crawling import get_29cm_product_info, get_29cm_product_reviews
from crawling_func.preprocess import remove_non_numeric
from crawling_func.normalized_category import search_category

import pandas as pd
from time import gmtime, strftime, sleep
import re

In [2]:
search_keywords = ["#선물추천", "선물언박싱", "생일선물언박싱", "#카카오톡선물하기", 
                   "부모님 선물", "남자 선물", "여자 선물", "남친선물", "여친선물",
                   "20대 선물", "30대 선물", "40대 선물", "50대 선물"]


In [26]:
# relevance_data = get_video_by_keys(search_keywords, 30)
current_data = get_video_by_keys(search_keywords, 80, order="date")

./YoutubeData/youtube_video_date_2024-07-09_10:40.csv 
총 201개의 영상이 수집되었습니다.


In [2]:
relevance_data = pd.read_csv("./YoutubeData/youtube_video_relevance_2024-07-08_13:09.csv", index_col=0)
current_data = pd.read_csv("./YoutubeData/youtube_video_date_2024-07-09_07:31.csv", index_col=0)

In [4]:
current_data

,id,searchQ,date,title,description,channel,tags,type,view
0,-XuDpEWLVtU,부모님 선물,2024-07-09T05:00:00Z,가수 박지현 결혼 30주년 부모님 선물 25억 원 상당 집 공개,가수 박지현 결혼 30주년 부모님 선물 25억 원 상당 집 공개,연예가소식,"['결혼', '이혼', '간통', '재혼', '재산', '임신', '황영웅', '임...",PT8M51S,130
1,-h4aR6_hZkc,남친선물,2024-03-20T10:30:00Z,남자친구 생일선물로 혼내주기 🎁✨ㅣ내돈내산 선물폭탄 언박싱 vlog,나한테 잘해라 박석땅 !\n\n\n▶석준이 인스타그램 : https://www.in...,석준이가해,"['석준이가해', '석준여름', '여름석준', '석준 여름', '여름 석준', '박...",PT11M6S,49282
2,-k_VCy27JZs,선물언박싱,2024-07-06T18:00:43Z,[어둠의전설]2024넥슨프라임 선물언박싱 ㄱㄱ,투네이션https://toon.at/donate/636864078271527343\...,준희형,"['어둠', '어둠의전설', '루루', '준희형', '넥슨', '메이플스토리', '...",PT2H55M38S,2118
3,-raEczrOaQ8,30대 선물,2024-03-30T14:42:00Z,(30대 여성 생일선물 EP5) 애플 에어팟 맥스 헤드폰,프로필 하단에 구매링크 클릭 가능합니다\nhttps://link.coupang.co...,TC생활건강 tv,[],PT1M,481
4,-y9cPfrwyos,"남자 선물, 여자 선물, 30대 선물, 40대 선물",2024-05-07T12:16:42Z,"선물 전문가의 5만원 내외 감도 높은 선물 추천 l 여자 선물, 남자 선물, 어른 ...",*이 영상은 유료광고를 포함하지 않습니다.\n*영상에서 소개한 제품/편집샵은 ‘더보...,챌린지유 challenzyu,"['선물추천', '선물', '남자선물', '집들이선물', '가정의달', '어른선물'...",PT17M2S,66032
...,...,...,...,...,...,...,...,...,...
198,y9_Nl3FOv7g,30대 선물,2024-03-31T12:54:21Z,VLOG l 30대 직장인 일상 브이로그 / 생일 선물 언박싱 / 포케 중독자 / ...,#직장인브이로그 #30대브이로그 \n\n촬영📹 아이폰14\n편집🎞️ 아이패드 에어3...,meet mii밋미,"['vlog', '브이로그', '직장인', '직장인브이로그', '30대', '30대...",PT19M3S,78
199,z4yvdcXxzEs,여자 선물,2024-07-02T02:30:00Z,[여자키] 선물 - 멜로망스 (KY.49594) / KY Karaoke,★ 곡 정보 ★\n번호 : 49594\n제목 : 선물\n가수 : 멜로망스\n작사 :...,금영 노래방 공식 유튜브 채널,"['금영', '금영엔터테인먼트', '노래방', '금영노래방', '노래방에서', '노...",PT4M23S,30
200,z665afdVckc,#카카오톡선물하기,2023-09-10T12:13:21Z,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...,#카카오톡선물하기 #생일선물추천 #집들이선물\n\nBusiness contact: ...,펄진주 PEARL,"['카카오톡 선물하기', '생일 선물 추천', '선물 추천 여자', '여자친구 선물...",PT19M50S,49702
201,zHvq4WO9iI4,#선물추천,2022-12-16T13:34:30Z,1~10만원대 센스있는 연말 선물 추천🎁 진짜 반응 좋은 생일&집들이 선물🥰,#선물추천 #연말선물 #집들이선물\n\n여러분 안녕하세요! 가영입니다 :)\n\n벌...,패션마케터 가영,"['집들이선물', '고급선물', '선물추천', '연말선물', '크리스마스선물', '...",PT17M2S,7609


In [5]:
relevance_k = (", ".join(list(relevance_data["searchQ"]))).split(", ")
current_k = (", ".join(list(current_data["searchQ"]))).split(", ")

count = [0,0]
for sk in search_keywords:
    print(f"{sk} : {relevance_k.count(sk)} / {current_k.count(sk)} ")
    count[0] += relevance_k.count(sk)
    count[1] += current_k.count(sk)

print(f"{len(relevance_k)} -> {count[0]} / {len(current_k)} -> {count[1]}")


#선물추천 : 24 / 24 
선물언박싱 : 25 / 19 
생일선물언박싱 : 25 / 25 
#카카오톡선물하기 : 20 / 16 
부모님 선물 : 15 / 19 
남자 선물 : 12 / 17 
여자 선물 : 4 / 18 
남친선물 : 19 / 13 
여친선물 : 17 / 6 
20대 선물 : 17 / 19 
30대 선물 : 17 / 21 
40대 선물 : 15 / 21 
50대 선물 : 23 / 22 
233 -> 233 / 240 -> 240


In [6]:
data = pd.concat([relevance_data, current_data])
data.reset_index(inplace=True, drop=True)
data = data.groupby("id").agg({"searchQ": ", ".join,
                                                 "date" : "first", 
                                                 "title" : "first", 
                                                 "description" : "first", 
                                                 "channel" : "first", 
                                                 "tags" : "first", 
                                                 "type" : "first", 
                                                 "view" : "first"})
data.reset_index(inplace=True)

data["searchQ"] = data["searchQ"].apply(lambda x: x.split(", "))
data

,id,searchQ,date,title,description,channel,tags,type,view
0,-XuDpEWLVtU,[부모님 선물],2024-07-09T05:00:00Z,가수 박지현 결혼 30주년 부모님 선물 25억 원 상당 집 공개,가수 박지현 결혼 30주년 부모님 선물 25억 원 상당 집 공개,연예가소식,"['결혼', '이혼', '간통', '재혼', '재산', '임신', '황영웅', '임...",PT8M51S,130
1,-h4aR6_hZkc,[남친선물],2024-03-20T10:30:00Z,남자친구 생일선물로 혼내주기 🎁✨ㅣ내돈내산 선물폭탄 언박싱 vlog,나한테 잘해라 박석땅 !\n\n\n▶석준이 인스타그램 : https://www.in...,석준이가해,"['석준이가해', '석준여름', '여름석준', '석준 여름', '여름 석준', '박...",PT11M6S,49282
2,-k_VCy27JZs,[선물언박싱],2024-07-06T18:00:43Z,[어둠의전설]2024넥슨프라임 선물언박싱 ㄱㄱ,투네이션https://toon.at/donate/636864078271527343\...,준희형,"['어둠', '어둠의전설', '루루', '준희형', '넥슨', '메이플스토리', '...",PT2H55M38S,2118
3,-raEczrOaQ8,[30대 선물],2024-03-30T14:42:00Z,(30대 여성 생일선물 EP5) 애플 에어팟 맥스 헤드폰,프로필 하단에 구매링크 클릭 가능합니다\nhttps://link.coupang.co...,TC생활건강 tv,[],PT1M,481
4,-y9cPfrwyos,"[30대 선물, 남자 선물, 여자 선물, 30대 선물, 40대 선물]",2024-05-07T12:16:42Z,"선물 전문가의 5만원 내외 감도 높은 선물 추천 l 여자 선물, 남자 선물, 어른 ...",*이 영상은 유료광고를 포함하지 않습니다.\n*영상에서 소개한 제품/편집샵은 ‘더보...,챌린지유 challenzyu,"['선물추천', '선물', '남자선물', '집들이선물', '가정의달', '어른선물'...",PT17M2S,65715
...,...,...,...,...,...,...,...,...,...
313,z665afdVckc,"[#카카오톡선물하기, #카카오톡선물하기]",2023-09-10T12:13:21Z,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...,#카카오톡선물하기 #생일선물추천 #집들이선물\n\nBusiness contact: ...,펄진주 PEARL,"['카카오톡 선물하기', '생일 선물 추천', '선물 추천 여자', '여자친구 선물...",PT19M50S,49689
314,zHvq4WO9iI4,[#선물추천],2022-12-16T13:34:30Z,1~10만원대 센스있는 연말 선물 추천🎁 진짜 반응 좋은 생일&집들이 선물🥰,#선물추천 #연말선물 #집들이선물\n\n여러분 안녕하세요! 가영입니다 :)\n\n벌...,패션마케터 가영,"['집들이선물', '고급선물', '선물추천', '연말선물', '크리스마스선물', '...",PT17M2S,7609
315,zKsuOHBD_xQ,"[#카카오톡선물하기, #카카오톡선물하기]",2023-10-15T11:50:00Z,(오늘의픽) [카카오톡 선물하기 추천] 언제까지 커피쿠폰 할거예요?ㅣ1-9만원대 가...,(영상은 1080 고화질로 보실 수 있습니다)\n\n** 이 영상은 유료광고를 포함...,오늘나래 onl narae,"['카카오톡 선물추천', '선물추천', '카카오 선물', '카카오 선물추천', '카...",PT13M32S,13877
316,zPZ4UdVeceU,[남친선물],2022-12-28T11:00:39Z,1만원~300만원 ‘센스있는‘ 남자친구 선물추천,#연말 #선물 \n─────────더보기란────────────\n(해당 영상은 협...,깡스타일리스트,"['깡스타일리스트', '선물', '속옷', '씨케이', '러쉬', '연말선물', '...",PT25M36S,201226


In [7]:
path = "./YoutubeData/youtube_video_final"+ strftime("%Y-%m-%d_%H:%M", gmtime()) + ".csv"
data.to_csv(path)


In [4]:
get_product_urls(data)

Failed to expand URL: https://link.coupang.com/a/bIkBUv -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/bIkBY7 -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/bIkBSS0 -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/bIkB3B -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/bFsW3u -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/bIBQcQ -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read timed out. (read timeout=5)
Failed to expand URL: https://link.coupang.com/a/6SiVJ -> HTTPSConnectionPool(host='www.coupang.com', port=443): Read

In [5]:
products = pd.read_csv("./YoutubeData/product_url_2024-07-09_14:23.csv", index_col=0)
products

,videoID,productURL
0,-y9cPfrwyos,https://mirabelle.shop/product/Alessi_LoveCoff...
1,-y9cPfrwyos,https://store.mmmg.kr/product/sb-bowl-light-br...
2,-y9cPfrwyos,https://www.lotteimall.com/goods/viewGoodsDeta...
3,-y9cPfrwyos,https://www.klalykke.com/goods/goods_view.php
4,-y9cPfrwyos,https://naturale.co.kr/goods/view
...,...,...
182,z665afdVckc,https://gift.kakao.com/product/7560932
183,z665afdVckc,https://gift.kakao.com/product/4463247
184,zHvq4WO9iI4,https://shop.dior.co.kr/products/y0996214
185,zHvq4WO9iI4,https://www.sivillage.com/goods/initDetailGood...


In [ ]:
products.drop_duplicates(subset=["productURL"],inplace=True)
products.reset_index(drop=True, inplace=True)
len(products)

In [8]:
products

,videoID,productURL
0,-y9cPfrwyos,https://mirabelle.shop/product/Alessi_LoveCoff...
1,-y9cPfrwyos,https://store.mmmg.kr/product/sb-bowl-light-br...
2,-y9cPfrwyos,https://www.lotteimall.com/goods/viewGoodsDeta...
3,-y9cPfrwyos,https://www.klalykke.com/goods/goods_view.php
4,-y9cPfrwyos,https://naturale.co.kr/goods/view
...,...,...
160,z665afdVckc,https://gift.kakao.com/product/4561890
161,z665afdVckc,https://gift.kakao.com/product/4424911
162,z665afdVckc,https://gift.kakao.com/product/7560932
163,z665afdVckc,https://gift.kakao.com/product/4463247


In [9]:
product_shoppingmall = []
for url in products['productURL']:
    if '29cm' in url : product_shoppingmall.append('29cm')
    elif any(shop in url for shop in ['gift.kakao', 'kko.to']): product_shoppingmall.append('카카오 선물하기')
    elif 'coupang' in url: product_shoppingmall.append('쿠팡')
    elif 'wconcept' in url: product_shoppingmall.append('wconcept')
    elif 'smartstore' in url: product_shoppingmall.append('네이버')
    elif any(shop in url for shop in ['sivillage', 'lotteon', 'hmall']): product_shoppingmall.append('백화점/홈쇼핑 몰')
    elif 'oliveyoung' in url: product_shoppingmall.append('올리브영')
    elif any(shop in url for shop in ['eqlstore', 'collecionb', 'ohou.se']): product_shoppingmall.append('인테리어/오브제 몰')
    elif 'kurly' in url: product_shoppingmall.append('마켓컬리')
    elif 'wconcept' in url: product_shoppingmall.append('29cm')
    elif 'makers.kakao' in url: product_shoppingmall.append('')
    else: product_shoppingmall.append('브랜드 공식 홈페이지')

products['shoppingmall'] = product_shoppingmall

In [10]:
products

,videoID,productURL,shoppingmall
0,-y9cPfrwyos,https://mirabelle.shop/product/Alessi_LoveCoff...,브랜드 공식 홈페이지
1,-y9cPfrwyos,https://store.mmmg.kr/product/sb-bowl-light-br...,브랜드 공식 홈페이지
2,-y9cPfrwyos,https://www.lotteimall.com/goods/viewGoodsDeta...,브랜드 공식 홈페이지
3,-y9cPfrwyos,https://www.klalykke.com/goods/goods_view.php,브랜드 공식 홈페이지
4,-y9cPfrwyos,https://naturale.co.kr/goods/view,브랜드 공식 홈페이지
...,...,...,...
160,z665afdVckc,https://gift.kakao.com/product/4561890,카카오 선물하기
161,z665afdVckc,https://gift.kakao.com/product/4424911,카카오 선물하기
162,z665afdVckc,https://gift.kakao.com/product/7560932,카카오 선물하기
163,z665afdVckc,https://gift.kakao.com/product/4463247,카카오 선물하기


In [11]:
products['shoppingmall'].value_counts()

shoppingmall
29cm           49
브랜드 공식 홈페이지    46
카카오 선물하기       36
쿠팡             20
wconcept        5
백화점/홈쇼핑 몰       4
인테리어/오브제 몰      2
                1
올리브영            1
마켓컬리            1
Name: count, dtype: int64

In [12]:
video_ids = []
urls = []
product_info = []
for idx, url in enumerate(products["productURL"]):
    if products["shoppingmall"][idx] == "카카오 선물하기":
        try: 
            temp_info =  get_kko_product_info(url)
            product_info.append(temp_info)
            urls.append(url)
            video_ids.append(products["videoID"][idx])
        except:
            print(f"excepted : {idx}")
            product_info.append(None)
            urls.append(url)
            video_ids.append(products["videoID"][idx])
    elif products["shoppingmall"][idx] == "29cm":
        try: 
            temp_info =  get_29cm_product_info(url)
            product_info.append(temp_info)
            urls.append(url)
            video_ids.append(products["videoID"][idx])
        except:
            print(f"excepted : {idx}")
            product_info.append(None)
            urls.append(url)
            video_ids.append(products["videoID"][idx])

https://product.29cm.co.kr/catalog/1732230 페이지 접근에서 문제 발생
https://product.29cm.co.kr/catalog/2050425 페이지 접근에서 문제 발생
https://gift.kakao.com/product/7737131 판매 중단
https://product.29cm.co.kr/catalog/1926189 페이지 접근에서 문제 발생
https://product.29cm.co.kr/catalog/1934886 페이지 접근에서 문제 발생
https://product.29cm.co.kr/catalog/1976669 페이지 접근에서 문제 발생
excepted : 82
https://gift.kakao.com/product/6983306 판매 중단
https://gift.kakao.com/product/6910140 판매 중단
excepted : 86
https://gift.kakao.com/product/2242277 판매 중단
https://gift.kakao.com/product/7394986 판매 중단
https://gift.kakao.com/product/7992874 판매 중단
https://gift.kakao.com/product/6218926 판매 중단
https://gift.kakao.com/product/7999434 판매 중단
https://gift.kakao.com/product/7869910 판매 중단
https://gift.kakao.com/product/6983293 판매 중단
excepted : 113
https://content.29cm.co.kr/post/41215 페이지 접근에서 문제 발생
https://product.29cm.co.kr/catalog/2051219 페이지 접근에서 문제 발생
https://gift.kakao.com/product/2318571 판매 중단
https://gift.kakao.com/product/4424911 페이지 접근에서 문제 발생
https:/

In [20]:
productDF = pd.DataFrame({"videoID":video_ids, "productURL":urls,"productInfo":product_info})
path = "./productInfo/product_final"+ strftime("%Y-%m-%d_%H:%M", gmtime()) + ".csv"
productDF.to_csv(path)

In [22]:
productDF.dropna(subset=["productInfo"], inplace=True)
productDF.reset_index(drop=True, inplace=True)
product_InfoDF = pd.DataFrame(columns=["videoID", "productURL", "product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category"])
for idx in range(len(productDF)):
    item = productDF["productInfo"].iloc[idx]
    item["videoID"] = productDF["videoID"].iloc[idx]
    item["productURL"] = productDF["productURL"].iloc[idx]
    try:
        tempDF = pd.DataFrame([item])[["videoID", "productURL", "product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category"]]      
    except:
        item["brand_description"] = None
        tempDF = pd.DataFrame([item])[["product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category"]]      
    product_InfoDF = pd.concat([product_InfoDF,tempDF])
product_InfoDF.reset_index(inplace=True, drop=True)
product_InfoDF
    

,videoID,productURL,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category
0,4vsC9nABHU8,https://product.29cm.co.kr/catalog/536616,"Lighthouse 유,무선 충전 포터블 조명 - 화이트","45,000원","45,000원",0%,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,https://shop.29cm.co.kr/brand/4189,https://img.29cm.co.kr/item/202310/11ee77d4240...,가전/생활가전/기타 생활가전
1,4vsC9nABHU8,https://product.29cm.co.kr/catalog/552913,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,"50,000","45,000원",10%,렉슨,1991년 설립된 디자인 생활용품 브랜드,https://shop.29cm.co.kr/brand/1235,https://img.29cm.co.kr/next-product/2022/02/04...,"가구,인테리어/조명/단스탠드"
2,4vsC9nABHU8,https://product.29cm.co.kr/catalog/2252385,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),"150,000원","150,000원",0%,헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/20523,https://img.29cm.co.kr/item/202309/11ee591b67b...,"가구,인테리어/조명"
3,8bEvMR2G2Zo,https://gift.kakao.com/product/1068105,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+...,"판매가 20,000 원","판매가 20,000 원",0%,캐스키드슨(뷰티),None,https://gift.kakao.com/brand/2722,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어
4,9CqqIRMlBgE,https://product.29cm.co.kr/catalog/889425,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],"63,000","38,900원",38%,알럽하우스,재미진 키친 알럽하우스,https://shop.29cm.co.kr/brand/3334,https://img.29cm.co.kr/next-product/2021/06/04...,"주방,생활/테이블웨어/유리컵"
...,...,...,...,...,...,...,...,...,...,...,...
57,z665afdVckc,https://gift.kakao.com/product/7391267,[괄사 스쿱 증정] '힐링 향 페이셜&바디 스크럽' 젤라또 팩스크럽 워시 210g...,"판매가 29,000 원","판매가 29,000 원",0%,휩드,None,https://gift.kakao.com/brand/14844,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어
58,z665afdVckc,https://gift.kakao.com/product/6836524,[선물하기 에디션] 부이 디지털 카메라 셀카/캠코더모드 지원(+SD카드) M Pr...,"판매가 36,900 원","판매가 36,900 원",0%,부이,None,https://gift.kakao.com/brand/15478,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카메라/캠코더/주변기기
59,z665afdVckc,https://gift.kakao.com/product/2356239,타르투플랑게 화이트 트러플 오일 + 소금 프리미엄 선물세트 (선물박스+쇼핑백),"판매가 52,000 원","판매가 52,000 원",0%,타르투플랑게,None,https://gift.kakao.com/brand/12874,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,가공식품
60,z665afdVckc,https://gift.kakao.com/product/4561890,[선물포장] 사포네 리퀴도 (핸드워시) 8종 택1,"판매가 82,000 원","판매가 82,000 원",0%,산타마리아노벨라,None,https://gift.kakao.com/brand/8597,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어


In [34]:
productDF
temp_data = data[["id", "searchQ", "title"]]
temp_data.columns = ["videoID", "searchQ", "title"]
product_InfoDF = pd.merge(product_InfoDF, temp_data, on="videoID", how="left")
product_InfoDF.columns = ["video_id", "product_url", "product_name", "original_price",
                          "current_price", "discount_rate", "brand_name", "brand_description",
                          "brand_link_inshop", "thumbnail", "category", "search_key", "video_title"]
product_InfoDF = product_InfoDF[["product_url", "product_name", 
                                 "original_price", "current_price", "discount_rate", 
                                 "brand_name", "brand_description", "brand_link_inshop", 
                                 "thumbnail", "category", 
                                 "search_key", "video_id", "video_title"]]
product_InfoDF["original_price"] = product_InfoDF["original_price"].apply(remove_non_numeric)
product_InfoDF["current_price"] = product_InfoDF["current_price"].apply(remove_non_numeric)
product_InfoDF
path = "./productInfo/product_final_fullInfo_"+ strftime("%Y-%m-%d_%H:%M", gmtime()) + ".csv"
product_InfoDF.to_csv(path)


In [35]:
product_InfoDF

,product_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category,search_key,video_id,video_title
0,https://product.29cm.co.kr/catalog/536616,"Lighthouse 유,무선 충전 포터블 조명 - 화이트",45000,45000,0%,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,https://shop.29cm.co.kr/brand/4189,https://img.29cm.co.kr/item/202310/11ee77d4240...,가전/생활가전/기타 생활가전,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
1,https://product.29cm.co.kr/catalog/552913,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,50000,45000,10%,렉슨,1991년 설립된 디자인 생활용품 브랜드,https://shop.29cm.co.kr/brand/1235,https://img.29cm.co.kr/next-product/2022/02/04...,"가구,인테리어/조명/단스탠드",['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
2,https://product.29cm.co.kr/catalog/2252385,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),150000,150000,0%,헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/20523,https://img.29cm.co.kr/item/202309/11ee591b67b...,"가구,인테리어/조명",['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
3,https://gift.kakao.com/product/1068105,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+...,20000,20000,0%,캐스키드슨(뷰티),None,https://gift.kakao.com/brand/2722,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,['#카카오톡선물하기'],8bEvMR2G2Zo,MZ껌뻑죽는 💛5만원 이하 카카오톡 선물하기 추천💛 뻔한거 말고 이거! (이사 집들...
4,https://product.29cm.co.kr/catalog/889425,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],63000,38900,38%,알럽하우스,재미진 키친 알럽하우스,https://shop.29cm.co.kr/brand/3334,https://img.29cm.co.kr/next-product/2021/06/04...,"주방,생활/테이블웨어/유리컵","['#선물추천', '#선물추천']",9CqqIRMlBgE,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,https://gift.kakao.com/product/7391267,[괄사 스쿱 증정] '힐링 향 페이셜&바디 스크럽' 젤라또 팩스크럽 워시 210g...,29000,29000,0%,휩드,None,https://gift.kakao.com/brand/14844,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
58,https://gift.kakao.com/product/6836524,[선물하기 에디션] 부이 디지털 카메라 셀카/캠코더모드 지원(+SD카드) M Pr...,36900,36900,0%,부이,None,https://gift.kakao.com/brand/15478,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카메라/캠코더/주변기기,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
59,https://gift.kakao.com/product/2356239,타르투플랑게 화이트 트러플 오일 + 소금 프리미엄 선물세트 (선물박스+쇼핑백),52000,52000,0%,타르투플랑게,None,https://gift.kakao.com/brand/12874,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,가공식품,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
60,https://gift.kakao.com/product/4561890,[선물포장] 사포네 리퀴도 (핸드워시) 8종 택1,82000,82000,0%,산타마리아노벨라,None,https://gift.kakao.com/brand/8597,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...


In [ ]:
product_info_lst = []
for idx in range(len(products)):
    sleep(5)
    product_name = products.product_name[idx]
    brand_name = products.brand_name[idx]
    product_info_lst.append(search_category(product_name, brand_name))
product_info_lst   

In [ ]:
temp = []
for item in product_info_lst:
    temp.append(re.sub(r"\\/","/",item))
    category = temp
cat1 = []
cat2 = []
cat3 = []
cat4 = []
for cat in category:
    try:
        splited = cat.split(" > ")  
    except:
        cat1.append(None)
        cat2.append(None)
        cat3.append(None)
        cat4.append(None)
        continue
    if len(splited) > 4: print("check this : cat")

    try:
        cat1.append(splited[0])
    except:
        cat1.append(None)
    try:
        cat2.append(splited[1])
    except:
        cat2.append(None)
    try:
        cat3.append(splited[2])
    except:
        cat3.append(None)
    try:
        cat4.append(splited[3])
    except:
        cat4.append(None)
        
categoryDF = pd.DataFrame({"대분류":cat1,"중분류":cat2,"소분류":cat3,"세분류":cat4})

print("대분류\n", categoryDF["대분류"].unique(),"\n")
print("중분류\n",categoryDF["중분류"].unique(),"\n")
print("소분류\n",categoryDF["소분류"].unique(),"\n")
print("세분류\n",categoryDF["세분류"].unique(),"\n")
categoryDF.groupby(list(categoryDF.columns)).count()

# DB에 넣을 데이터

In [ ]:
product_InfoDF.columns

Index(['product_url', 'product_name', 'original_price', 'current_price',
       'discount_rate', 'brand_name', 'brand_description', 'brand_link_inshop',
       'thumbnail', 'category', 'search_key', 'video_id', 'video_title'],
      dtype='object')

In [ ]:
product_db = product_InfoDF[["product_name", "brand_name", "brand_description", 
                       "original_price", "current_price", "discount_rate",
                       "product_url", "thumbnail"]]

In [ ]:
product_db["discount_rate"] = product_db.discount_rate.apply(lambda x: int(re.sub("%","",x)))
product_db["product_name"] = product_db.product_name.apply(lambda x: re.sub(",","/",x).strip())
product_db["shoppingmall"] = product_db.product_url.apply(lambda x: "29cm" if "29cm" in x else ("카카오톡 선물하기" if "kakao" in x else None))

In [ ]:
cat_db = pd.DataFrame(columns=["name", "level","parent"])
# cat_types = list(category_group.columns)
# cat_types
new_catDF = categoryDF[["대분류", "중분류", "소분류"]].sort_values(by=["대분류", "중분류", "소분류"]).drop_duplicates()
new_catDF

for cat in new_catDF["대분류"].unique():
    cat_db = pd.concat([cat_db, pd.DataFrame({"name" : [cat], "level": ["대분류"],"parent" : [None]})],ignore_index=True)

columns = new_catDF.columns
for c_idx in range(1,len(columns)):
    tempDF = new_catDF[columns[:c_idx+1]].drop_duplicates().reset_index(drop=True)
    for idx in range(len(tempDF)):
        level = columns[c_idx]
        cat = tempDF[level][idx]
        if cat == "" or cat is None: continue
        parent = tempDF[columns[c_idx-1]][idx]
        cat_db = pd.concat([cat_db, pd.DataFrame({"name" : [cat], "level": [level],"parent" : [parent]})],ignore_index=True)

cat_db.to_csv("./productInfo/db_category(1).csv")

In [ ]:
product_db["category"] = product_info_lst

/var/folders/sx/jpkxvsy509b318mtl_2tcqq40000gn/T/ipykernel_67168/3603955860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_db["category"] = product_info_lst


In [ ]:
product_db = pd.read_csv("./productInfo/db_product(1).csv",index_col=0)
product_db["category"] = product_info_lst
product_db.to_csv("./productInfo/db_data(1).csv")

# DB수정 및 One!t 컬렉션 데이터 추가

In [3]:
db = pd.read_csv("./productInfo/db_product(1).csv", index_col=0)

In [4]:
db.head()

,product_name,brand_name,brand_description,original_price,current_price,discount_rate,product_url,thumbnail,shoppingmall,category
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,45000,45000,0,https://product.29cm.co.kr/catalog/536616,https://img.29cm.co.kr/item/202310/11ee77d4240...,29cm,가구/인테리어
1,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,렉슨,1991년 설립된 디자인 생활용품 브랜드,50000,45000,10,https://product.29cm.co.kr/catalog/552913,https://img.29cm.co.kr/next-product/2022/02/04...,29cm,가구/인테리어
2,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,150000,150000,0,https://product.29cm.co.kr/catalog/2252385,https://img.29cm.co.kr/item/202309/11ee591b67b...,29cm,가구/인테리어
3,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+배...,캐스키드슨(뷰티),NaN,20000,20000,0,https://gift.kakao.com/product/1068105,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,화장품/미용
4,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],알럽하우스,재미진 키친 알럽하우스,63000,38900,38,https://product.29cm.co.kr/catalog/889425,https://img.29cm.co.kr/next-product/2021/06/04...,29cm,생활/건강


In [27]:
rawDF = pd.read_csv("./productInfo/product_final_fullInfo_2024-07-10_03:34.csv", index_col=0)
rawDF

,product_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category,search_key,video_id,video_title
0,https://product.29cm.co.kr/catalog/536616,"Lighthouse 유,무선 충전 포터블 조명 - 화이트",45000,45000,0%,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,https://shop.29cm.co.kr/brand/4189,https://img.29cm.co.kr/item/202310/11ee77d4240...,가전/생활가전/기타 생활가전,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
1,https://product.29cm.co.kr/catalog/552913,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,50000,45000,10%,렉슨,1991년 설립된 디자인 생활용품 브랜드,https://shop.29cm.co.kr/brand/1235,https://img.29cm.co.kr/next-product/2022/02/04...,"가구,인테리어/조명/단스탠드",['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
2,https://product.29cm.co.kr/catalog/2252385,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),150000,150000,0%,헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/20523,https://img.29cm.co.kr/item/202309/11ee591b67b...,"가구,인테리어/조명",['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
3,https://gift.kakao.com/product/1068105,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+...,20000,20000,0%,캐스키드슨(뷰티),NaN,https://gift.kakao.com/brand/2722,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,['#카카오톡선물하기'],8bEvMR2G2Zo,MZ껌뻑죽는 💛5만원 이하 카카오톡 선물하기 추천💛 뻔한거 말고 이거! (이사 집들...
4,https://product.29cm.co.kr/catalog/889425,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],63000,38900,38%,알럽하우스,재미진 키친 알럽하우스,https://shop.29cm.co.kr/brand/3334,https://img.29cm.co.kr/next-product/2021/06/04...,"주방,생활/테이블웨어/유리컵","['#선물추천', '#선물추천']",9CqqIRMlBgE,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,https://gift.kakao.com/product/7391267,[괄사 스쿱 증정] '힐링 향 페이셜&바디 스크럽' 젤라또 팩스크럽 워시 210g...,29000,29000,0%,휩드,NaN,https://gift.kakao.com/brand/14844,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
58,https://gift.kakao.com/product/6836524,[선물하기 에디션] 부이 디지털 카메라 셀카/캠코더모드 지원(+SD카드) M Pr...,36900,36900,0%,부이,NaN,https://gift.kakao.com/brand/15478,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카메라/캠코더/주변기기,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
59,https://gift.kakao.com/product/2356239,타르투플랑게 화이트 트러플 오일 + 소금 프리미엄 선물세트 (선물박스+쇼핑백),52000,52000,0%,타르투플랑게,NaN,https://gift.kakao.com/brand/12874,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,가공식품,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
60,https://gift.kakao.com/product/4561890,[선물포장] 사포네 리퀴도 (핸드워시) 8종 택1,82000,82000,0%,산타마리아노벨라,NaN,https://gift.kakao.com/brand/8597,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...


In [28]:
rawDF = pd.concat([db, rawDF[["brand_link_inshop","search_key","video_id","video_title"]]],axis=1)

In [20]:
rawDF

,product_name,brand_name,brand_description,original_price,current_price,discount_rate,product_url,thumbnail,shoppingmall,category,brand_link_inshop,search_key,video_id,video_title
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,45000,45000,0,https://product.29cm.co.kr/catalog/536616,https://img.29cm.co.kr/item/202310/11ee77d4240...,29cm,가구/인테리어,https://shop.29cm.co.kr/brand/4189,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
1,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,렉슨,1991년 설립된 디자인 생활용품 브랜드,50000,45000,10,https://product.29cm.co.kr/catalog/552913,https://img.29cm.co.kr/next-product/2022/02/04...,29cm,가구/인테리어,https://shop.29cm.co.kr/brand/1235,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
2,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,150000,150000,0,https://product.29cm.co.kr/catalog/2252385,https://img.29cm.co.kr/item/202309/11ee591b67b...,29cm,가구/인테리어,https://shop.29cm.co.kr/brand/20523,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편
3,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+배...,캐스키드슨(뷰티),NaN,20000,20000,0,https://gift.kakao.com/product/1068105,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,화장품/미용,https://gift.kakao.com/brand/2722,['#카카오톡선물하기'],8bEvMR2G2Zo,MZ껌뻑죽는 💛5만원 이하 카카오톡 선물하기 추천💛 뻔한거 말고 이거! (이사 집들...
4,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],알럽하우스,재미진 키친 알럽하우스,63000,38900,38,https://product.29cm.co.kr/catalog/889425,https://img.29cm.co.kr/next-product/2021/06/04...,29cm,생활/건강,https://shop.29cm.co.kr/brand/3334,"['#선물추천', '#선물추천']",9CqqIRMlBgE,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,[괄사 스쿱 증정] '힐링 향 페이셜&바디 스크럽' 젤라또 팩스크럽 워시 210g ...,휩드,NaN,29000,29000,0,https://gift.kakao.com/product/7391267,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,화장품/미용,https://gift.kakao.com/brand/14844,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
58,[선물하기 에디션] 부이 디지털 카메라 셀카/캠코더모드 지원(+SD카드) M Pro 디카,부이,NaN,36900,36900,0,https://gift.kakao.com/product/6836524,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,디지털/가전,https://gift.kakao.com/brand/15478,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
59,타르투플랑게 화이트 트러플 오일 + 소금 프리미엄 선물세트 (선물박스+쇼핑백),타르투플랑게,NaN,52000,52000,0,https://gift.kakao.com/product/2356239,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,식품,https://gift.kakao.com/brand/12874,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...
60,[선물포장] 사포네 리퀴도 (핸드워시) 8종 택1,산타마리아노벨라,NaN,82000,82000,0,https://gift.kakao.com/product/4561890,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,화장품/미용,https://gift.kakao.com/brand/8597,"['#카카오톡선물하기', '#카카오톡선물하기']",z665afdVckc,선물 고민 멈춰!💥센스 만점 카카오톡 선물하기 11가지 추천⭐️ (생일 선물 추천/...


In [12]:
product_info_lst = []
for idx in range(len(rawDF)):
    product_name = rawDF.product_name[idx]
    brand_name = rawDF.brand_name[idx]
    product_info_lst.append(search_category(product_name, brand_name))
    
    
# path = "./productInfo/product"+ strftime("%Y-%m-%d_%H:%M", gmtime()) + ".csv"

# products.to_csv(path)

product_info_lst

Error : No Results -> 비건 팩클렌저 디스커버리 키트 + 팝업 카드 + 팩스크럽 3종 샘플키트 증정
-> find product success!
Error : No Results -> 슬밋 퍼퓸 핸드크림 30ml
Error : No Results -> 슬밋(뷰티) 슬밋 퍼퓸 핸드크림 30ml
-> find product success!
Error : No Results -> 밀로 로프티 스퀘어 퀼팅 패디드 숄더 토트백
Error : No Results -> 밀로 로프티 스퀘어 퀼팅 패디드 숄더 토트백
-> find product success!
Error : No Results -> 블몬 Y2K 레트로 셀피 캠코더 비디오 카메라
Error : No Results -> 블몬 Y2K 레트로 셀피 캠코더 비디오 카메라
-> find product success!
Error : No Results -> 부이 디지털 카메라 셀카/캠코더모드 지원 M Pro 디카
-> find product success!


['가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명',
 '가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명',
 '가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명',
 '화장품/미용 > 바디케어 > 핸드케어 > ',
 '생활/건강 > 주방용품 > 잔/컵 > 유리컵',
 '가구/인테리어 > 카페트/러그 > 발매트 > ',
 '디지털/가전 > 생활가전 > 구강청정기 > 칫솔살균기',
 '디지털/가전 > 휴대폰액세서리 > 휴대폰충전기 > 충전기',
 '생활/건강 > 주방용품 > 잔/컵 > 유리컵',
 '생활/건강 > 주방용품 > 잔/컵 > 머그',
 '생활/건강 > 주방용품 > 보관/밀폐용기 > 도시락통/찬합',
 '식품 > 건강식품 > 꿀 > ',
 '식품 > 음료 > 차류 > 허브차',
 '식품 > 과자/베이커리 > 캐러멜 > ',
 '생활/건강 > 문구/사무용품 > 노트/수첩 > 노트',
 '패션잡화 > 여성신발 > 실내화 > ',
 '생활/건강 > 주방용품 > 식기 > 양식기',
 '패션잡화 > 여성가방 > 파우치 > ',
 '패션잡화 > 패션소품 > 키홀더 > ',
 '가구/인테리어 > 인테리어소품 > 화병 > ',
 '패션잡화 > 모자 > 야구모자 > 일반캡',
 '식품 > 과자/베이커리 > 기타과자 > ',
 '생활/건강 > 주방용품 > 식기 > 양식기',
 '패션잡화 > 여성신발 > 슬리퍼 > ',
 '가구/인테리어 > 인테리어소품 > 스탠드 > 단스탠드',
 '가구/인테리어 > 인테리어소품 > 스탠드 > 단스탠드',
 '화장품/미용 > 클렌징 > 클렌징크림 > ',
 '화장품/미용 > 바디케어 > 핸드케어 > ',
 '화장품/미용 > 스킨케어 > 미스트 > ',
 '화장품/미용 > 바디케어 > 바디케어세트 > ',
 '생활/건강 > 주방용품 > 식기 > 볼',
 '패션잡화 > 여성가방 > 토트백 > ',
 '패션잡화 > 모자 > 야구모자 > 일반캡',
 '패션잡화 > 여성가방 > 숄더백 > ',
 '출산/육아 > 인형 > 봉제인

In [29]:
rawDF["display_category"] = product_info_lst

In [30]:
rawDF.columns

Index(['product_name', 'brand_name', 'brand_description', 'original_price',
       'current_price', 'discount_rate', 'product_url', 'thumbnail',
       'shoppingmall', 'category', 'brand_link_inshop', 'search_key',
       'video_id', 'video_title', 'display_category'],
      dtype='object')

In [31]:
rawrawDF = rawrawDF[["product_name", "brand_name", "brand_description", "original_price",
        "current_price", "discount_rate", "product_url", "thumbnail",
        "shoppingmall", "brand_link_inshop", "search_key",
        "video_id", "video_title", "display_category"]]
rawrawDF["large_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[0])
rawrawDF["middle_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[1])
rawDF["small_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[2])
rawDF.head()

,product_name,brand_name,brand_description,original_price,current_price,discount_rate,product_url,thumbnail,shoppingmall,brand_link_inshop,search_key,video_id,video_title,display_category,large_category,middle_category,small_category
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,45000,45000,0,https://product.29cm.co.kr/catalog/536616,https://img.29cm.co.kr/item/202310/11ee77d4240...,29cm,https://shop.29cm.co.kr/brand/4189,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
1,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,렉슨,1991년 설립된 디자인 생활용품 브랜드,50000,45000,10,https://product.29cm.co.kr/catalog/552913,https://img.29cm.co.kr/next-product/2022/02/04...,29cm,https://shop.29cm.co.kr/brand/1235,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
2,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,150000,150000,0,https://product.29cm.co.kr/catalog/2252385,https://img.29cm.co.kr/item/202309/11ee591b67b...,29cm,https://shop.29cm.co.kr/brand/20523,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
3,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+배...,캐스키드슨(뷰티),NaN,20000,20000,0,https://gift.kakao.com/product/1068105,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,https://gift.kakao.com/brand/2722,['#카카오톡선물하기'],8bEvMR2G2Zo,MZ껌뻑죽는 💛5만원 이하 카카오톡 선물하기 추천💛 뻔한거 말고 이거! (이사 집들...,화장품/미용 > 바디케어 > 핸드케어 >,화장품/미용,바디케어,핸드케어
4,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],알럽하우스,재미진 키친 알럽하우스,63000,38900,38,https://product.29cm.co.kr/catalog/889425,https://img.29cm.co.kr/next-product/2021/06/04...,29cm,https://shop.29cm.co.kr/brand/3334,"['#선물추천', '#선물추천']",9CqqIRMlBgE,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ...",생활/건강 > 주방용품 > 잔/컵 > 유리컵,생활/건강,주방용품,잔/컵


In [32]:
rawDF.to_csv("./productInfo/raw_product(2).csv")      

In [33]:
rawDF[["product_name", "brand_name", "brand_description", "original_price",
    "product_url", "thumbnail",
    "shoppingmall", "display_category","large_category","middle_category", "small_category"]].to_csv("./productInfo/db_product(2).csv")
    

In [2]:
collections_df= pd.read_csv("./productInfo/human_tagging - 워닛 컬렉션.csv")
collections_df = collections_df[["product_url","shoppingmall"]][:54]
collections_df["product_url"] = collections_df.product_url.apply(lambda x: re.sub("\?","",x))
collections_df["shoppingmall"] = collections_df.shoppingmall.apply(lambda x: "카카오 선물하기" if "카카오" in x else x)
collections_df

In [10]:
video_ids = ["NO VIDEO - Oneit Collection"] * len(collections_df)
searh_keys = [None] * len(collections_df)
product_info = []
for idx, url in enumerate(collections_df["product_url"]):
    if collections_df["shoppingmall"][idx] == "카카오 선물하기":
        try: 
            temp_info =  get_kko_product_info(url)
            product_info.append(temp_info)
        except:
            print(f"excepted : {idx}")
            product_info.append(None)
    elif collections_df["shoppingmall"][idx] == "29cm":
        try: 
            temp_info =  get_29cm_product_info(url)
            product_info.append(temp_info)
        except:
            print(f"excepted : {idx}")
            product_info.append(None)

https://product.29cm.co.kr/catalog/2431457 페이지 접근에서 문제 발생
https://gift.kakao.com/product/9829728 판매 중단


In [26]:
productDF = pd.DataFrame({"video_id":video_ids, "search_key": searh_keys,"productInfo":product_info, "product_url" :collections_df.product_url[:54]})
productDF.dropna(subset=["productInfo"], inplace=True)
productDF.reset_index(drop=True, inplace=True)
product_InfoDF = pd.DataFrame(columns=["video_id", "product_url", "product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category", "search_key"])
for idx in range(len(productDF)):
    item = productDF["productInfo"].iloc[idx]
    item["video_id"] = productDF["video_id"].iloc[idx]
    item["search_key"] = productDF["search_key"].iloc[idx]
    item["product_url"] = productDF["product_url"].iloc[idx]
    try:
        tempDF = pd.DataFrame([item])[["video_id", "product_url", "product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category", "search_key"]]      
    except:
        item["brand_description"] = None
        tempDF = pd.DataFrame([item])[["video_id", "product_url", "product_name", "original_price", "current_price", "discount_rate", "brand_name", "brand_description", "brand_link_inshop", "thumbnail", "category", "search_key"]]      
    product_InfoDF = pd.concat([product_InfoDF,tempDF])
product_InfoDF.reset_index(inplace=True, drop=True)

    

In [28]:
product_InfoDF.head()

,video_id,product_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category,search_key
0,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2015265,M풀탭 밴드 맥세이프 아이폰 갤럭시 전기종 호환 마그네틱 탈부착 가죽/레더 핸드폰 ...,"24,900","22,410원",10%,신지모루,편리한 쓰임새와 간결한 디자인의 모바일 액세서리,https://shop.29cm.co.kr/brand/9380,https://img.29cm.co.kr/next-product/2023/04/13...,"컴퓨터,디지털/모바일 액세서리/기타 액세서리",None
1,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1820362,컴피코지 v.2 랩탑 백 13인치 4color,"44,800원","44,800원",0%,라이브워크,None,https://shop.29cm.co.kr/brand/7647,https://img.29cm.co.kr/next-product/2022/11/11...,남성가방/랩탑백,None
2,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2566762,Apple 맥세이프 아이폰 파인우븐 카드지갑 퍼시픽 블루 MT263FE/A,"85,000","70,975원",17%,Apple,모두가 사랑하는 Apple,https://shop.29cm.co.kr/brand/16404,https://img.29cm.co.kr/item/202404/11eef0b968e...,"컴퓨터,디지털/모바일 액세서리/스마트폰 케이스",None
3,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848622,[공식]_척 70 클래식 블랙 / 162058C,"95,000","85,500원",10%,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/30...,남성신발/스니커즈/로우탑,None
4,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848606,[공식]_척 70 클래식 블랙 / 162050C,"99,000","89,100원",10%,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/22...,여성신발/스니커즈/하이탑,None


In [27]:
product_InfoDF.columns

Index(['video_id', 'product_url', 'product_name', 'original_price',
       'current_price', 'discount_rate', 'brand_name', 'brand_description',
       'brand_link_inshop', 'thumbnail', 'category', 'search_key'],
      dtype='object')

In [29]:
product_InfoDF["video_title"] = [None] * len(product_InfoDF)
product_InfoDF["shoppingmall"] = collections_df.shoppingmall[:54]


In [ ]:
rawDF = rawDF[["product_name", "brand_name", "brand_description", "original_price",
        "current_price", "discount_rate", "product_url", "thumbnail",
        "shoppingmall", "brand_link_inshop", "search_key",
        "video_id", "video_title", "display_category"]]
rawDF["large_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[0])
rawDF["middle_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[1])
rawDF["small_category"] = rawDF.display_category.apply(lambda x: x.split(" > ")[2])
rawDF.head()

In [30]:
product_InfoDF.head()

,video_id,product_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category,search_key,video_title,shoppingmall
0,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2015265,M풀탭 밴드 맥세이프 아이폰 갤럭시 전기종 호환 마그네틱 탈부착 가죽/레더 핸드폰 ...,"24,900","22,410원",10%,신지모루,편리한 쓰임새와 간결한 디자인의 모바일 액세서리,https://shop.29cm.co.kr/brand/9380,https://img.29cm.co.kr/next-product/2023/04/13...,"컴퓨터,디지털/모바일 액세서리/기타 액세서리",None,None,29cm
1,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1820362,컴피코지 v.2 랩탑 백 13인치 4color,"44,800원","44,800원",0%,라이브워크,None,https://shop.29cm.co.kr/brand/7647,https://img.29cm.co.kr/next-product/2022/11/11...,남성가방/랩탑백,None,None,29cm
2,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2566762,Apple 맥세이프 아이폰 파인우븐 카드지갑 퍼시픽 블루 MT263FE/A,"85,000","70,975원",17%,Apple,모두가 사랑하는 Apple,https://shop.29cm.co.kr/brand/16404,https://img.29cm.co.kr/item/202404/11eef0b968e...,"컴퓨터,디지털/모바일 액세서리/스마트폰 케이스",None,None,29cm
3,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848622,[공식]_척 70 클래식 블랙 / 162058C,"95,000","85,500원",10%,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/30...,남성신발/스니커즈/로우탑,None,None,29cm
4,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848606,[공식]_척 70 클래식 블랙 / 162050C,"99,000","89,100원",10%,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/22...,여성신발/스니커즈/하이탑,None,None,29cm


In [32]:
product_info_lst = []

for idx in range(len(product_InfoDF)):
    sleep(5)
    product_name = product_InfoDF.product_name[idx]
    brand_name = product_InfoDF.brand_name[idx]
    product_info_lst.append(search_category(product_name, brand_name))
product_info_lst   

Error : No Results -> 메종드코튼 그리니/블랙키/블루이 호텔타올 코마 40수 190g 2매
-> find product success!
Error : No Results -> 에이센트 오브제 스톤디퓨저ㅣ방향제/인테리어/화산석/디퓨져/선물/결혼
-> find product success!
Error : No Results -> 더 스프레이 450ml - 100% 제주 편백 탈취 정화수
-> find product success!
Error : No Results -> 라포즈 필로우미스트 + 양면안대 청량 숲속 우디
-> find product success!
Error : No Results -> 립 포션 바미로즈 + 크림치크+스크런치
-> find product success!


['디지털/가전 > 휴대폰액세서리 > 휴대폰케이스 > 가죽케이스',
 '디지털/가전 > 노트북액세서리 > 노트북가방/케이스 > ',
 '디지털/가전 > 휴대폰액세서리 > 기타휴대폰액세서리 > ',
 '패션잡화 > 남성신발 > 운동화 > 캔버스화',
 '패션잡화 > 남성신발 > 운동화 > 하이탑',
 '식품 > 과자/베이커리 > 초콜릿 > ',
 '화장품/미용 > 뷰티소품 > 바디소품 > ',
 '패션잡화 > 여행용가방/소품 > 여행소품케이스 > ',
 '생활/건강 > 눈건강용품 > 안대 > ',
 '패션잡화 > 패션소품 > 양산 > 수동양산',
 '패션잡화 > 주얼리 > 목걸이 > 패션목걸이',
 '생활/건강 > 자동차용품 > 공기청정용품 > 차량용방향제',
 '패션잡화 > 여행용가방/소품 > 보스턴가방 > ',
 '가구/인테리어 > 침구단품 > 홑이불 > ',
 '패션잡화 > 남성신발 > 운동화 > 러닝화',
 '생활/건강 > 생활용품 > 화장지 > 화장지케이스',
 '식품 > 주류 > 전통주 > 전통주선물세트',
 '디지털/가전 > 휴대폰액세서리 > 휴대폰충전기 > 충전기',
 '디지털/가전 > 이미용가전 > 면도기 > 남성용면도기',
 '디지털/가전 > 음향가전 > 스피커 > 블루투스스피커',
 '생활/건강 > 문구/사무용품 > 다이어리/플래너 > 캘린더/달력',
 '디지털/가전 > 생활가전 > 구강청정기 > 칫솔살균기',
 '디지털/가전 > 이미용가전 > 고데기 > ',
 '식품 > 음료 > 차류 > 기타차',
 '가구/인테리어 > 인테리어소품 > 아로마/캔들용품 > 아로마방향제/디퓨저',
 '패션잡화 > 여성가방 > 백팩 > ',
 '패션잡화 > 여성신발 > 부츠 > 미들부츠',
 '패션잡화 > 주얼리 > 반지 > 패션반지',
 '디지털/가전 > 주변기기 > 키보드 > 무선키보드',
 '생활/건강 > 욕실용품 > 수건/타월 > 세면타월',
 '가구/인테리어 > 인테리어소품 > 아로마/캔들용품 > 아로마방향제/디퓨저',
 '디지털/가전 > 음향가전 > 

In [38]:
product_InfoDF["display_category"] = product_info_lst
product_InfoDF["large_category"] = product_InfoDF.display_category.apply(lambda x: x.split(" > ")[0])
product_InfoDF["middle_category"] = product_InfoDF.display_category.apply(lambda x: x.split(" > ")[1])
product_InfoDF["small_category"] = product_InfoDF.display_category.apply(lambda x: x.split(" > ")[2])

product_InfoDF["original_price"] = product_InfoDF["original_price"].apply(remove_non_numeric)
product_InfoDF["current_price"] = product_InfoDF["current_price"].apply(remove_non_numeric)
product_InfoDF["discount_rate"] = product_InfoDF["discount_rate"].apply(remove_non_numeric)
product_InfoDF["search_key"] = product_InfoDF["search_key"].apply(lambda x: [])

product_InfoDF.head()

,video_id,product_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_link_inshop,thumbnail,category,search_key,video_title,shoppingmall,display_category,large_category,middle_category,small_category
0,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2015265,M풀탭 밴드 맥세이프 아이폰 갤럭시 전기종 호환 마그네틱 탈부착 가죽/레더 핸드폰 ...,24900,22410,10,신지모루,편리한 쓰임새와 간결한 디자인의 모바일 액세서리,https://shop.29cm.co.kr/brand/9380,https://img.29cm.co.kr/next-product/2023/04/13...,"컴퓨터,디지털/모바일 액세서리/기타 액세서리",[],None,29cm,디지털/가전 > 휴대폰액세서리 > 휴대폰케이스 > 가죽케이스,디지털/가전,휴대폰액세서리,휴대폰케이스
1,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1820362,컴피코지 v.2 랩탑 백 13인치 4color,44800,44800,0,라이브워크,None,https://shop.29cm.co.kr/brand/7647,https://img.29cm.co.kr/next-product/2022/11/11...,남성가방/랩탑백,[],None,29cm,디지털/가전 > 노트북액세서리 > 노트북가방/케이스 >,디지털/가전,노트북액세서리,노트북가방/케이스
2,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/2566762,Apple 맥세이프 아이폰 파인우븐 카드지갑 퍼시픽 블루 MT263FE/A,85000,70975,17,Apple,모두가 사랑하는 Apple,https://shop.29cm.co.kr/brand/16404,https://img.29cm.co.kr/item/202404/11eef0b968e...,"컴퓨터,디지털/모바일 액세서리/스마트폰 케이스",[],None,29cm,디지털/가전 > 휴대폰액세서리 > 기타휴대폰액세서리 >,디지털/가전,휴대폰액세서리,기타휴대폰액세서리
3,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848622,[공식]_척 70 클래식 블랙 / 162058C,95000,85500,10,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/30...,남성신발/스니커즈/로우탑,[],None,29cm,패션잡화 > 남성신발 > 운동화 > 캔버스화,패션잡화,남성신발,운동화
4,NO VIDEO - Oneit Collection,https://product.29cm.co.kr/catalog/1848606,[공식]_척 70 클래식 블랙 / 162050C,99000,89100,10,컨버스,자유로움의 표상,https://shop.29cm.co.kr/brand/1154,https://img.29cm.co.kr/next_product/2022/12/22...,여성신발/스니커즈/하이탑,[],None,29cm,패션잡화 > 남성신발 > 운동화 > 하이탑,패션잡화,남성신발,운동화


In [37]:
tot_df = pd.read_csv("./productInfo/raw_product(2).csv",index_col=0)
tot_df.columns

Index(['product_name', 'brand_name', 'brand_description', 'original_price',
       'current_price', 'discount_rate', 'product_url', 'thumbnail',
       'shoppingmall', 'brand_link_inshop', 'search_key', 'video_id',
       'video_title', 'display_category', 'large_category', 'middle_category',
       'small_category'],
      dtype='object')

In [39]:
col_df = product_InfoDF[['product_name', 'brand_name', 'brand_description', 'original_price',
                                 'current_price', 'discount_rate', 'product_url', 'thumbnail',
                                 'shoppingmall', 'brand_link_inshop', 'search_key', 'video_id',
                                 'video_title', 'display_category', 'large_category', 'middle_category',
                                 'small_category']]

In [41]:
tot_df = pd.concat([tot_df,col_df])
tot_df.reset_index(inplace=True, drop=True)


,product_name,brand_name,brand_description,original_price,current_price,discount_rate,product_url,thumbnail,shoppingmall,brand_link_inshop,search_key,video_id,video_title,display_category,large_category,middle_category,small_category
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,밀레니얼 세대의 첫 취향 소비,45000,45000,0,https://product.29cm.co.kr/catalog/536616,https://img.29cm.co.kr/item/202310/11ee77d4240...,29cm,https://shop.29cm.co.kr/brand/4189,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
1,LEXON 렉슨 MINA S사이즈 미나 무드등 조명 램프 - LH60,렉슨,1991년 설립된 디자인 생활용품 브랜드,50000,45000,10,https://product.29cm.co.kr/catalog/552913,https://img.29cm.co.kr/next-product/2022/02/04...,29cm,https://shop.29cm.co.kr/brand/1235,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
2,HAY 피씨 포터블 PC Portable (Anthracite/Soft Black),헤이,북유럽 디자인의 감각을 담은 리빙 & 라이프스타일 브랜드,150000,150000,0,https://product.29cm.co.kr/catalog/2252385,https://img.29cm.co.kr/item/202309/11ee591b67b...,29cm,https://shop.29cm.co.kr/brand/20523,['여친선물'],4vsC9nABHU8,가격대별 여자친구/여사친 선물 총정리 종결편,가구/인테리어 > 인테리어소품 > 조명 > 인테리어조명,가구/인테리어,인테리어소품,조명
3,[단독/쇼핑백 증정] 캐스키드슨 킵카인드 핸드&바디 4종세트(워시+로션+핸드크림+배...,캐스키드슨(뷰티),NaN,20000,20000,0,https://gift.kakao.com/product/1068105,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오톡 선물하기,https://gift.kakao.com/brand/2722,['#카카오톡선물하기'],8bEvMR2G2Zo,MZ껌뻑죽는 💛5만원 이하 카카오톡 선물하기 추천💛 뻔한거 말고 이거! (이사 집들...,화장품/미용 > 바디케어 > 핸드케어 >,화장품/미용,바디케어,핸드케어
4,디즈니정품 글라스 빈티지 유리컵 4P [Premium 레트로컬렉션],알럽하우스,재미진 키친 알럽하우스,63000,38900,38,https://product.29cm.co.kr/catalog/889425,https://img.29cm.co.kr/next-product/2021/06/04...,29cm,https://shop.29cm.co.kr/brand/3334,"['#선물추천', '#선물추천']",9CqqIRMlBgE,"5만 원 이하 센스 있는 선물 추천! 🎁 29CM 선물하기 / 집들이 선물, 자취 ...",생활/건강 > 주방용품 > 잔/컵 > 유리컵,생활/건강,주방용품,잔/컵
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,슈틸루스터 퀵차지 고속 맥세이프 보조배터리 5000mAh ST-PBM5000,슈틸루스터,None,35900,35900,0,https://gift.kakao.com/product/8751389,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오 선물하기,https://gift.kakao.com/brand/13993,[],NO VIDEO - Oneit Collection,None,디지털/가전 > 휴대폰액세서리 > 휴대폰배터리 > 보조배터리,디지털/가전,휴대폰액세서리,휴대폰배터리
110,맥세이프 보조배터리 아이폰 무선충전 10000mAh (맥세이프 미니 보조배터리 M...,만렙,None,39500,39500,0,https://gift.kakao.com/product/7304117,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오 선물하기,https://gift.kakao.com/brand/15879,[],NO VIDEO - Oneit Collection,None,디지털/가전 > 휴대폰액세서리 > 휴대폰배터리 > 보조배터리,디지털/가전,휴대폰액세서리,휴대폰배터리
111,[롯데백화점]UA 언디나이어블 5.0 스몰 더플 백 1369222-001,언더아머,None,58500,58500,0,https://gift.kakao.com/product/9997131,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오 선물하기,https://gift.kakao.com/brand/3636,[],NO VIDEO - Oneit Collection,None,스포츠/레저 > 축구 > 축구가방 > 필드용,스포츠/레저,축구,축구가방
112,[에이오유] 매트 포슬밤,헤메코,None,17800,17800,0,https://gift.kakao.com/product/9132216,https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카카오 선물하기,https://gift.kakao.com/brand/16799,[],NO VIDEO - Oneit Collection,None,화장품/미용 > 색조메이크업 > 립틴트 >,화장품/미용,색조메이크업,립틴트


In [97]:
tot_df[68:71]

,product_name,brand_name,brand_description,original_price,current_price,discount_rate,product_url,thumbnail,shoppingmall,brand_link_inshop,search_key,video_id,video_title,display_category,large_category,middle_category,small_category
68,피카소X함경식 메이크업 스파츌라+미니 믹싱 팔레트 기획세트,피카소,"나를 위한 뷰티툴, 피부에 닿는 '극강의 부드러운 터치'를 위해 태어난 아티스트 감...",26000,19900,23,https://product.29cm.co.kr/catalog/2266504,https://img.29cm.co.kr/item/202309/11ee5cd2bca...,29cm,https://shop.29cm.co.kr/brand/10108,[],NO VIDEO - Oneit Collection,None,화장품/미용 > 뷰티소품 > 바디소품,화장품/미용,뷰티소품,바디소품
69,멀티 캐리어 파우치(+듀얼미러 증정),피카소,"나를 위한 뷰티툴, 피부에 닿는 '극강의 부드러운 터치'를 위해 태어난 아티스트 감...",29000,26100,10,https://product.29cm.co.kr/catalog/2273335,https://img.29cm.co.kr/item/202310/11ee634b5c2...,29cm,https://shop.29cm.co.kr/brand/10108,[],NO VIDEO - Oneit Collection,None,패션잡화 > 여행용가방/소품 > 여행소품케이스,패션잡화,여행용가방/소품,여행소품케이스
70,[선물포장] 슬립타이트 필로우 미스트 100ML & 수면 안대 세트,아로마티카,피부도 살리고 지구도 살리는 클린 & 비건 뷰티 브랜드,30000,27000,10,https://product.29cm.co.kr/catalog/1842161,https://img.29cm.co.kr/item/202403/11eee1d44bb...,29cm,https://shop.29cm.co.kr/brand/1503,[],NO VIDEO - Oneit Collection,None,생활/건강 > 눈건강용품 > 안대,생활/건강,눈건강용품,안대


In [98]:
tot_df["product_name"] = tot_df["product_name"].apply(lambda x: re.sub(",", " ", x))
temp_lst = []
for b in tot_df.brand_description:
    try:
        temp_lst.append(re.sub(",", " ",b))
    except:
        temp_lst.append(b)
tot_df["brand_description"] =temp_lst
tot_df["display_category"] = tot_df["display_category"].apply(lambda x: x[:-3] if x[-3:]==" > " else x)
tot_df.to_csv("./productInfo/raw_product(2).csv")

In [131]:
tot_df["gender"] = tagDF["gender"]

In [132]:
tot_df.columns

Index(['product_name', 'brand_name', 'brand_description', 'original_price',
       'current_price', 'discount_rate', 'product_url', 'thumbnail',
       'shoppingmall', 'brand_link_inshop', 'search_key', 'video_id',
       'video_title', 'display_category', 'large_category', 'middle_category',
       'small_category', 'gender'],
      dtype='object')

In [134]:
tot_df[['product_name', 'brand_name', 'brand_description', 'original_price',
        'current_price', 'discount_rate','shoppingmall',
        'product_url', 'thumbnail', 
        'display_category', 'large_category', 'middle_category', 'small_category','gender']].to_csv("./productInfo/db_product(2).csv")

In [101]:
categoryDF = pd.read_csv("./productInfo/db_product(2).csv",index_col=0)
categoryDF=categoryDF[["large_category", "middle_category","small_category"]]
categoryDF

,large_category,middle_category,small_category
0,가구/인테리어,인테리어소품,조명
1,가구/인테리어,인테리어소품,조명
2,가구/인테리어,인테리어소품,조명
3,화장품/미용,바디케어,핸드케어
4,생활/건강,주방용품,잔/컵
...,...,...,...
109,디지털/가전,휴대폰액세서리,휴대폰배터리
110,디지털/가전,휴대폰액세서리,휴대폰배터리
111,스포츠/레저,축구,축구가방
112,화장품/미용,색조메이크업,립틴트


In [109]:
new_catDF[90:]

,large_category,middle_category,small_category


In [74]:
".".join(["가", "나"])

'가.나'

In [108]:
cat_db = pd.DataFrame(columns=["name", "level","parent"])
# cat_types = list(category_group.columns)
# cat_types
new_catDF = categoryDF[["large_category", "middle_category","small_category"]].sort_values(by=["large_category", "middle_category","small_category"]).drop_duplicates()
new_catDF

for cat in new_catDF["large_category"].unique():
    cat_db = pd.concat([cat_db, pd.DataFrame({"name" : [cat], "level": ["large_category"],"parent" : [None], "display_name": [cat]})],ignore_index=True)

columns = new_catDF.columns
for c_idx in range(1,len(columns)):
    tempDF = new_catDF[columns[:c_idx+1]].drop_duplicates().reset_index(drop=True)
    print(tempDF)
    for idx in range(len(tempDF)):
        level = columns[c_idx]
        cat = tempDF[level][idx]
        if cat == "" or cat is None: continue
        parent = tempDF[columns[c_idx-1]][idx]
        # print(parent,cat)
        try:
            display_name = " > ".join([parent, cat])
        except:
            display_name = parent
        cat_db = pd.concat([cat_db, pd.DataFrame({"name" : [cat], "level": [level],"parent" : [parent], "display_name": [display_name]})],ignore_index=True)


   large_category middle_category
0         가구/인테리어          인테리어소품
1         가구/인테리어            침구단품
2         가구/인테리어          카페트/러그
3          디지털/가전            계절가전
4          디지털/가전         노트북액세서리
5          디지털/가전            생활가전
6          디지털/가전            음향가전
7          디지털/가전           이미용가전
8          디지털/가전            주방가전
9          디지털/가전            주변기기
10         디지털/가전       카메라/캠코더용품
11         디지털/가전         휴대폰액세서리
12          생활/건강           눈건강용품
13          생활/건강         문구/사무용품
14          생활/건강           발건강용품
15          생활/건강            생활용품
16          생활/건강            욕실용품
17          생활/건강              음반
18          생활/건강           자동차용품
19          생활/건강            주방용품
20         스포츠/레저              축구
21             식품            건강식품
22             식품         과자/베이커리
23             식품          식용유/오일
24             식품              음료
25             식품            잼/시럽
26             식품             조미료
27             식품              주류
28          출산

In [111]:
# cat_db["level"] = cat_db["level"].apply(lambda x: "대분류" if x == "large_category" else x)
# cat_db["level"] = cat_db["level"].apply(lambda x: "중분류" if x == "middle_category" else x)
# cat_db["level"] = cat_db["level"].apply(lambda x: "소분류" if x == "small_category" else x)
cat_db

,name,level,parent,display_name
0,가구/인테리어,대분류,None,가구/인테리어
1,디지털/가전,대분류,None,디지털/가전
2,생활/건강,대분류,None,생활/건강
3,스포츠/레저,대분류,None,스포츠/레저
4,식품,대분류,None,식품
...,...,...,...,...
138,미스트,소분류,스킨케어,스킨케어 > 미스트
139,클렌징크림,소분류,클렌징,클렌징 > 클렌징크림
140,남녀공용향수,소분류,향수,향수 > 남녀공용향수
141,여성향수,소분류,향수,향수 > 여성향수


In [112]:
cat_db.to_csv("./productInfo/db_category(2).csv")

In [126]:
tagDF = pd.read_csv('./productInfo/human_tagging.csv',index_col=0)

In [127]:
tagDF.reset_index(inplace=True)

In [128]:
tagDF.columns

Index(['product_name', 'brand_name', 'brand_description', 'original_price',
       'product_url', 'shoppingmall', 'large', 'middle', 'small', 'gender',
       'color', 'style', 'active', 'practicality', 'taste', 'cooking',
       'cosmetic', 'incense', 'display_category'],
      dtype='object')

In [118]:
tagDF = tagDF[['product_name', 'brand_name',
                'color', 'style', 'active', 'practicality', 'taste', 'cooking',
                'cosmetic', 'incense']]

In [123]:
keywords = ['color', 'style', 'active', 'practicality', 'taste', 'cooking','cosmetic', 'incense']
resultDF  = pd.DataFrame(columns=["product_name", "brand_name", "keyword"])
for idx in range(len(tagDF)):
    for k in keywords:
        # print(pd.DataFrame({"product_name" : [resultDF.product_name[idx]], 
        #                                               "brand_name": [resultDF.brand_name[idx]], 
        #                                               "keyword" : [resultDF[k][idx]]}))
        resultDF = pd.concat([resultDF, pd.DataFrame({"product_name" : [tagDF.product_name[idx]], 
                                                      "brand_name": [tagDF.brand_name[idx]], 
                                                      "keyword" : [tagDF[k][idx]]})])
resultDF

,product_name,brand_name,keyword
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,무채색
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,심플
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,집
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,디자인
0,Lighthouse 유/무선 충전 포터블 조명 - 화이트,모바일아일랜드,NaN
...,...,...,...
0,지샥 GA-2100-1A1DR,지샥,실용성
0,지샥 GA-2100-1A1DR,지샥,NaN
0,지샥 GA-2100-1A1DR,지샥,NaN
0,지샥 GA-2100-1A1DR,지샥,NaN


In [124]:
resultDF.to_csv("./productInfo/tags.csv")

In [130]:
tagDF["gender"].to_csv("./productInfo/gender.csv")

In [ ]:
tot_df["gender"] = tagDF["gender"]
tot_df.to_csv("./productInfo/raw_product(2).csv")